In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import Twitter, Kkma
import numpy as np
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, GRU, Dropout, Conv1D, MaxPooling1D
from tqdm import tqdm_notebook as tq
from gensim.summarization import summarizer
from sklearn.feature_extraction.text import TfidfVectorizer
tq().pandas()

Using TensorFlow backend.


In [58]:
df1 = pd.read_csv('data/생산_제조.csv',engine='python', index_col=0)
df2 = pd.read_csv('data/IT.csv',engine='python', index_col=0)
df3 = pd.read_csv('data/경영사무.csv',engine='python', index_col=0)
df4 = pd.read_csv('data/무역유통.csv',engine='python', index_col=0)
df5 = pd.read_csv('data/영업_고객상담.csv',engine='python', index_col=0)

In [59]:
df1.tail()

,회사,질문,답,직무
1,S-OIL㈜,귀하가 지원한 직무를 지원한 동기가 무엇이고 희망직무와 관련된 본인의 경험 또는 노...,"\n답변\n\n**식품, 택배 등 다양한 아르바이트를 하며 제품 생산에서부터 포장,...",생산_제조
2,S-OIL㈜,"대학생활 중 가장 뛰어난 성과를 거두었던 경험이 무엇이었으며, 그 과정에서 어떠한 ...","\n답변\n\n[`함께`라는 의미]\n 올해 4월, 현재 소속된 봉사단 `Y-smu...",생산_제조
3,S-OIL㈜,"귀하가 경험했던 가장 어려웠던 일이 무엇이었으며, 그것을 극복하기 위하여 어떤 노력...",\n답변\n\n1학년 때 가입한 동아리 `새앎터`가 있습니다. 하지만 군대에 있을 ...,생산_제조
4,S-OIL㈜,"타인과의 협업 과정에서 나타나는 본인의 장점 및 단점, 단점을 극복하기 위한 노력 ...","\n답변\n\n동아리 회장, 봉사활동 팀장 다양한 직책을 맡았습니다. 정기적인 회의...",생산_제조
5,S-OIL㈜,"소위 말하는 스펙(학교, 학점, 전공, 어학점수 등)을 제외하고 S-OIL이 귀하를...",\n답변\n\nSpecialized\n`생산자동화` 과제로 생산라인을 만들었습니다....,생산_제조


In [60]:
df = pd.concat([df1,df2,df3,df4,df5])

In [61]:
df = df.reset_index()

In [62]:
df['직무']=df['직무'].replace('생산_제조',1)
df['직무']=df['직무'].replace('IT',2)
df['직무']=df['직무'].replace('경영_사무',3)
df['직무']=df['직무'].replace('무역_유통',4)
df['직무']=df['직무'].replace('영업_고객상담',5)

In [63]:
df['답']=df['답'].str.replace("\n"," ")
df['답']=df['답'].str.replace("답변","")
df['답']=df['답'].str.replace('"',"")
df['답']=df['답'].str.replace('아쉬운점',"")
df['답']=df['답'].str.replace('좋은점',"")
df['답']=df['답'].str.replace('1',"")
df['답']=df['답'].str.replace('2',"")
df['답']=df['답'].str.replace('3',"")
df['답']=df['답'].str.replace('4',"")
df['답']=df['답'].str.replace('5',"")
df['답']=df['답'].str.replace('자',"")
df['답']=df['답'].str.replace('Byte',"")
df['답']=df['답'].str.replace('#',"")


In [64]:
np.unique(df['직무'])

array([1, 2, 3, 4, 5], dtype=int64)

In [65]:
df = df[df['답'].apply(len)>500]
len(df)

11250

In [66]:
#df['Answer'] = df['답'].copy()
#
#for i in tq(range(len(df['Answer']))):
#    try:
#        for j in [0.8,0.6,0.4,0.2]:
#            summ = []
#            summ = summarizer.summarize(df['답'].iloc[i],ratio=j)
#            if(len(summ)<=100):
#                df['Answer'].iloc[i] = summ
#                break
#    except ValueError:
#        pass

In [67]:
df['Answer'] = df['답'].copy()

for i in tq(range(len(df['Answer']))):
    try:
        summ = []
        summ = summarizer.summarize(df['답'].iloc[i],word_count=100)
        df['Answer'].iloc[i] = summ
    except ValueError:
        pass

C:\Users\smhrd08\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [68]:
kkma = Kkma()
twitter = Twitter()

C:\Users\smhrd08\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)
C:\Users\smhrd08\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [69]:
df['Answer1'] = df['Answer'].progress_apply(kkma.nouns)

In [70]:
df['Answer2'] = df['Answer'].progress_apply(twitter.nouns)

In [71]:
df['Answer3'] = np.nan
for i in tq(range(len(df))):
    df['Answer3'].iloc[i] = list(set(df['Answer1'].iloc[i]).intersection(df['Answer2'].iloc[i]))

In [72]:
df = pd.read_csv('asdf.csv',engine='python', index_col=0)

In [73]:
X_data = df['Answer3'].copy()
y_data = df['직무']

In [74]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data) 
sequences = tokenizer.texts_to_sequences(X_data) 
print(sequences[:2])

[[325, 59, 118, 51, 375, 396, 339, 308, 969, 141, 3, 1504, 278, 89, 336, 137, 23, 303, 194, 1437, 290, 48, 8, 471, 430, 714, 1562, 37, 304, 205, 221, 228, 475, 567, 60, 21, 145, 111, 163, 489, 79, 698, 27, 38, 109, 64, 748, 208, 348, 280, 419, 168, 13, 1, 32, 17, 16, 547, 14, 1281, 946, 24, 238, 10, 70, 469, 1624, 803, 1760, 2636, 92, 326, 36, 71, 19, 5, 40, 314, 11, 1132, 121, 286, 226, 3962, 994, 523, 18, 125], [305, 51, 375, 3963, 2502, 141, 3, 1504, 89, 1783, 405, 108, 303, 726, 102, 1437, 76, 8, 354, 135, 1984, 1562, 445, 104, 1864, 686, 551, 174, 228, 96, 235, 567, 376, 206, 60, 95, 65, 489, 1713, 251, 29, 79, 698, 38, 144, 748, 208, 2569, 1485, 2570, 8207, 86, 3805, 1, 32, 43, 491, 52, 2398, 1438, 424, 184, 886, 201, 412, 53, 676, 25, 670, 374, 662, 985, 2956, 1056, 185, 1270, 10, 146, 70, 49, 41, 768, 961, 169, 67, 1760, 197, 2783, 379, 69, 595, 19, 7, 5, 40, 1607, 105, 573, 107, 47, 1761, 254, 1865, 1034, 359, 258, 151, 302, 431, 523, 18]]


In [75]:
word_to_index = tokenizer.word_index
print(max(word_to_index.values()))

14528


In [76]:
maxlen = max(len(l) for l in X_data)
print('최대 길이',maxlen)
print('평균 길이 : %f' % (sum(map(len, X_data))/len(X_data)))

최대 길이 2054
평균 길이 : 276.978207


In [77]:
X_train, X_test, y_train, y_test = train_test_split(sequences,y_data.tolist(), test_size=0.3, random_state=1234,shuffle=True, stratify= y_data.values)

In [78]:
print(len(X_train),len(y_train))

9379 9379


In [79]:
X_train_seq = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test_seq = sequence.pad_sequences(X_test, maxlen=maxlen)

In [80]:
y_train_ctg = np_utils.to_categorical(y_train)
y_test_ctg = np_utils.to_categorical(y_test)

In [81]:
lstm_model = Sequential()
lstm_model.add(Embedding(max(word_to_index.values())+1, 100,input_length=maxlen))
lstm_model.add(Conv1D(filters=32,kernel_size=5,padding="same",activation='relu'))
lstm-model.add(MaxPooling1D(pool_size=2))
lstm_model.add(LSTM(100, activation='relu'))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(20, activation='relu'))
lstm_model.add(Dense(6, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
lstm_history = lstm_model.fit(X_train_seq, y_train_ctg, verbose=2,
                    epochs=20,validation_data=(X_test_seq, y_test_ctg))

Train on 9379 samples, validate on 4020 samples
Epoch 1/20
 - 575s - loss: 1.3892 - accuracy: 0.4407 - val_loss: 1.1868 - val_accuracy: 0.5527
Epoch 2/20


In [ ]:
#gru_model = Sequential()
#gru_model.add(Embedding(17257, 1000))
#gru_model.add(GRU(1000, activation='tanh'))
#gru_model.add(Dense(100, activation='linear'))
#gru_model.add(Dense(6, activation='softmax'))
#gru_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#gru_history = gru_model.fit(X_train_seq, y_train_ctg, batch_size=64, 
#                    epochs=20,validation_data=(X_test_seq, y_test_ctg))